In [147]:
import pandas as pd
import numpy as np
import pickle as p

In [148]:
data_all = pd.read_csv('../data_all.csv')

# Test with Subset of Data
np.random.seed(43)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=50, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

# # Test with All Data
# data_subset = data_all.copy()

In [149]:
# item_subset

### `/train`

In [150]:
import time

In [151]:
import requests
import json
from threading import Thread

url = 'https://fillet.azurewebsites.net/train/'

data_subset.to_parquet('data_subset.parquet')
files = {'data': open('data_subset.parquet', 'rb')}

payload = {'cv_acc': True, 'project_id': 2}

Thread(target=requests.post, args=(url, ), kwargs={'data': payload, 'files': files}).start()

time.sleep(10)
files['data'].close()

### `/query_progress`

In [157]:
import requests
import json

url = 'https://fillet.azurewebsites.net/query_progress/'


payload = {'project_id': 2}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=1200)

print('% Complete:',round(r.json()['pct_complete'],3))
print('CV Done:',r.json()['cv_done'])

% Complete: 78.0
CV Done: 0


### `/get_cv_results`

In [144]:
import requests
import json

url = 'https://fillet.azurewebsites.net/get_cv_results/'


payload = {'project_id': 2}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=1200)
pd.DataFrame().from_dict((json.loads(r.json())))

,item_id,avg_sales,r2_score,mae_score,mpe_score,rmse_score
0,3732.0,126.142509,0.667140,24.869097,0.197151,32.708429
1,3766.0,91.448227,0.679326,18.620106,0.203614,24.921558
2,16019.0,109.613182,0.351513,30.572566,0.278913,38.491338
3,3726.0,74.383606,0.650015,16.124501,0.216775,21.172948
4,3743.0,132.480972,0.715015,29.602702,0.223449,39.776576
5,2589.0,161.616043,0.674510,34.653754,0.214420,46.441378
6,2531.0,318.983582,0.552831,71.550972,0.224309,93.539758
7,1230.0,8.996674,0.357139,3.886082,0.431947,5.479785
8,1195.0,129.492813,0.551499,31.307370,0.241769,39.545600
9,202.0,2110.440186,0.734149,343.470402,0.162748,475.105621


### `/optimize`

In [ ]:
url = 'http://localhost:5000/optimize/'

payload = {'project_id': 1,
           'constraints': [],
           'population': 100,
           'max_epoch': 200
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [ ]:
best_prices = r.json()['result']
best_prices

In [ ]:
import requests
import json

url = 'http://localhost:5000/predict/'
payload = {'prices': best_prices,
           'project_id': 1
           }
headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }
payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

# What If Prediction
qty = r.json()['qty_estimates']

In [ ]:
qty

In [ ]:
# Calculate Revenue
revenue = 0
for item in best_prices.keys():
    revenue += (best_prices[item]*qty['Qty_'+item])
    
print(revenue)

### `/predict/`

In [146]:
# Sample Actual Price 
from helper_functions import *

sales_data = data_subset

sales_data = optimize_memory(sales_data)

sales_data_wide = sales_data.set_index(
    ['Wk', 'Tier', 'Store', 'Item_ID']).unstack(level=-1).reset_index().copy()
sales_data_wide.columns = [
    ''.join(str(i) for i in col).strip()
    for col in sales_data_wide.columns.values
]
sales_data_wide = sales_data_wide.sort_values(
    ['Tier', 'Store', 'Wk'], ascending=True).reset_index(drop=True)

sales_data_wide_clean = sales_data_wide.dropna(axis=0).copy()
dataset = sales_data_wide_clean
price_columns = [
    col for col in sales_data_wide_clean.columns if col.startswith('Price')
]

a = dataset.sample(1)
a_dict = a[price_columns].iloc[0].to_dict()
a_input = {}
for key in a_dict.keys():
    a_input[key.split('_')[1]]=round(a_dict[key],2)

print('Input Prices Quantities:')
# a_input


# Predict Sales Qty and Compare to Actual

import requests
import json

url = 'https://fillet.azurewebsites.net/predict/'

payload = {'prices': a_input, 'project_id': 2}

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)
qty = r.json()['qty_estimates']
qty_outp = {}
for key in qty.keys():
    qty_outp[key] = round(qty[key], 0)
actual = a[[
    col for col in sales_data_wide_clean.columns if col.startswith('Qty_')
]].iloc[0].to_dict()
pd.DataFrame([qty_outp, actual], index=['Estimated', 'Actual']).transpose()

,Estimated,Actual
Qty_1195,223.0,195.0
Qty_1230,19.0,17.0
Qty_16019,210.0,210.0
Qty_202,5193.0,4785.0
Qty_2531,517.0,703.0
Qty_2589,419.0,277.0
Qty_3726,243.0,254.0
Qty_3732,342.0,350.0
Qty_3743,210.0,242.0
Qty_3766,241.0,227.0
